In [2]:
!pip install transformers torch datasets

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 542.0/542.0 kB 9.1 MB/s eta 0:00:00
  Using cached nvidia_cuda_nvrtc_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (23.7 MB)
  Using cached nvidia_cuda_runtime_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (823 kB)
  Using cached nvidia_cuda_cupti_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (14.1 MB)
  Using cached nvidia_cudnn_cu12-8.9.2.26-py3-none-manylinux1_x86_64.whl (731.7 MB)
  Using cached nvidia_cublas_cu12-12.1.3.1-py3-none-manylinux1_x86_64.whl (410.6 MB)
  Using cached nvidia_cufft_cu12-11.0.2.54-py3-none-manylinux1_x86_64.whl (121.6 MB)
  Using cached nvidia_curand_cu12-10.3.2.106-py3-none-manylinux1_x86_64.whl (56.5 MB)
  Using cached nvidia_cusolver_cu12-11.4.5.107-py3-none-manylinux1_x86_64.whl (124.2 MB)
  Using cached nvidia_cusparse_cu12-12.1.0.106-py3-none-manylinux1_x86_64.whl (196.0 MB)
  Using cached nvidia_nccl_cu12-2.19.3-py3-none-manylinux1_x86_64.whl (166.0 MB)
  Using cached nvidia_nvtx_cu12-12.1.105-py3-none-many

In [3]:
!pip install accelerate -U


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 297.6/297.6 kB 7.0 MB/s eta 0:00:00


In [4]:
!pip install rouge_score sacrebleu


  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 106.7/106.7 kB 4.2 MB/s eta 0:00:00
  Created wheel for rouge_score: filename=rouge_score-0.1.2-py3-none-any.whl size=24933 sha256=09e90e319562f61c89246306f06817d63350af93e87072b24df82044f36d2aa8
  Stored in directory: /root/.cache/pip/wheels/5f/dd/89/461065a73be61a532ff8599a28e9beef17985c9e9c31e541b4
Successfully built rouge_score


In [1]:
from transformers import T5Tokenizer, T5ForConditionalGeneration, Trainer, TrainingArguments
from transformers import DataCollatorForSeq2Seq, EarlyStoppingCallback
from datasets import load_dataset
import numpy as np
import torch

In [7]:
# Function to load model and tokenizer from Hugging Face Model Hub
def load_model():
    model_name = "t5-base"  # Use T5-base model
    tokenizer = T5Tokenizer.from_pretrained(model_name)
    model = T5ForConditionalGeneration.from_pretrained(model_name)
    return tokenizer, model

In [8]:
# Function to load and prepare data
def load_and_prepare_data(tokenizer):
    dataset = load_dataset("neulab/tldr")
    train_test_split = dataset["train"].train_test_split(test_size=0.2)
    dataset["train"] = train_test_split["train"]
    dataset["validation"] = train_test_split["test"]

    def preprocess_function(examples):
        input_texts = ["translate English to Bash: " + inp for inp in examples['nl']]
        model_inputs = tokenizer(input_texts, max_length=30, truncation=True, padding="max_length")
        with tokenizer.as_target_tokenizer():
            labels = tokenizer(examples['cmd'], max_length=30, truncation=True, padding="max_length")
        model_inputs['labels'] = labels['input_ids']
        return model_inputs

    tokenized_datasets = dataset.map(preprocess_function, batched=True)
    return tokenized_datasets

# Load model and tokenizer
tokenizer, model = load_model()

# Load and prepare data
tokenized_datasets = load_and_prepare_data(tokenizer)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


spiece.model:   0%|          | 0.00/792k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.39M [00:00<?, ?B/s]

config.json:   0%|          | 0.00/1.21k [00:00<?, ?B/s]

You are using the default legacy behaviour of the <class 'transformers.models.t5.tokenization_t5.T5Tokenizer'>. This is expected, and simply means that the `legacy` (previous) behavior will be used so nothing changes for you. If you want to use the new behaviour, set `legacy=False`. This should only be set if you understand what it means, and thoroughly read the reason why this was added as explained in https://github.com/huggingface/transformers/pull/24565
Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


...

Generating validation split: 0 examples [00:00, ? examples/s]

Map:   0%|          | 0/5131 [00:00<?, ? examples/s]

/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:3921: UserWarning: `as_target_tokenizer` is deprecated and will be removed in v5 of Transformers. You can tokenize your labels by using the argument `text_target` of the regular `__call__` method (either in the same call as your input texts if you use the same keyword arguments, or in a separate call.
  warnings.warn(


Map:   0%|          | 0/928 [00:00<?, ? examples/s]

Map:   0%|          | 0/1283 [00:00<?, ? examples/s]

In [4]:
from google.colab import drive
drive.mount('/content/drive')
model_name = "t5-base"
# model_path = '/content/drive/My Drive/linux_command_translator_base'
# tokenizer, model = load_model(model_path)


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [5]:
# Training setup
training_args = TrainingArguments(
    output_dir="./results",
    evaluation_strategy="epoch",
    learning_rate=2e-5,
    per_device_train_batch_size=1,
    per_device_eval_batch_size=1,
    gradient_accumulation_steps=16,
    num_train_epochs=5,
    weight_decay=0.01,
    save_strategy="epoch",
    save_total_limit=3,
    load_best_model_at_end=True,
    metric_for_best_model='bleu',
    report_to="none",
    fp16=True
)

In [10]:
import os
from rouge_score import rouge_scorer, scoring
from sacrebleu.metrics import BLEU
from datasets import load_metric
import numpy as np

# Load metrics for evaluation
bleu_metric = load_metric('sacrebleu')
rouge_metric = load_metric('rouge')

def compute_metrics(eval_pred):
    logits, labels = eval_pred

    # Check if logits is a tuple and extract appropriately
    if isinstance(logits, tuple):
        logits = logits[0]

    # Convert to numpy arrays if they are tensors
    if isinstance(logits, torch.Tensor):
        logits = logits.detach().cpu().numpy()

    if isinstance(labels, torch.Tensor):
        labels = labels.detach().cpu().numpy()

    # Perform argmax to get predictions
    predictions = np.argmax(logits, axis=-1)

    # Decode the predictions and labels using the tokenizer
    decoded_preds = tokenizer.batch_decode(predictions, skip_special_tokens=True)
    decoded_labels = tokenizer.batch_decode(labels, skip_special_tokens=True)

    # Clean up tokens
    decoded_preds = [pred.strip() for pred in decoded_preds]
    decoded_labels = [[label.strip()] for label in decoded_labels]

    # Calculate metrics
    bleu_score = bleu_metric.compute(predictions=decoded_preds, references=decoded_labels)['score']
    rouge_result = rouge_metric.compute(predictions=decoded_preds, references=decoded_labels)
    rouge_l_score = rouge_result['rougeL'].mid.fmeasure

    # Calculate perplexity if possible
    # Directly compute perplexity from logits if losses are not separately provided
    if logits is not None:
        # Assume cross-entropy loss for perplexity calculation
        softmax_logits = torch.nn.functional.softmax(torch.tensor(logits), dim=-1)
        label_ids = torch.tensor(labels)
        # Use gather to pull probabilities of actual labels
        gathered_probs = softmax_logits.gather(dim=-1, index=label_ids.unsqueeze(-1)).squeeze(-1)
        mean_log_prob = gathered_probs.log().mean().item()
        perplexity = np.exp(-mean_log_prob)
    else:
        perplexity = float('nan')

    return {'bleu': bleu_score, 'rouge-l': rouge_l_score, 'perplexity': perplexity}


<ipython-input-10-cd5c73119686>:8: FutureWarning: load_metric is deprecated and will be removed in the next major version of datasets. Use 'evaluate.load' instead, from the new library 🤗 Evaluate: https://huggingface.co/docs/evaluate
  bleu_metric = load_metric('sacrebleu')
/usr/local/lib/python3.10/dist-packages/datasets/load.py:759: FutureWarning: The repository for sacrebleu contains custom code which must be executed to correctly load the metric. You can inspect the repository content at https://raw.githubusercontent.com/huggingface/datasets/2.19.0/metrics/sacrebleu/sacrebleu.py
You can avoid this message in future by passing the argument `trust_remote_code=True`.
Passing `trust_remote_code=True` will be mandatory to load this metric from the next major release of `datasets`.
  warnings.warn(


/usr/local/lib/python3.10/dist-packages/datasets/load.py:759: FutureWarning: The repository for rouge contains custom code which must be executed to correctly load the metric. You can inspect the repository content at https://raw.githubusercontent.com/huggingface/datasets/2.19.0/metrics/rouge/rouge.py
You can avoid this message in future by passing the argument `trust_remote_code=True`.
Passing `trust_remote_code=True` will be mandatory to load this metric from the next major release of `datasets`.
  warnings.warn(


In [11]:
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_datasets['train'],
    eval_dataset=tokenized_datasets['validation'],
    data_collator=DataCollatorForSeq2Seq(tokenizer),
    compute_metrics=compute_metrics,
    callbacks=[EarlyStoppingCallback(early_stopping_patience=3)]
)
# Start training
trainer.train()

Epoch,Training Loss,Validation Loss,Bleu,Rouge-l,Perplexity
0,No log,1.421982,12.757783,0.186430,4.145331
1,2.098300,1.336141,13.915776,0.206973,3.804308
2,2.098300,1.297673,14.505073,0.220523,3.660751
3,1.400700,1.279497,14.950140,0.228263,3.594813


Epoch,Training Loss,Validation Loss,Bleu,Rouge-l,Perplexity
0,No log,1.421982,12.757783,0.186430,4.145331
1,2.098300,1.336141,13.915776,0.206973,3.804308
2,2.098300,1.297673,14.505073,0.220523,3.660751
3,1.400700,1.279497,14.950140,0.228263,3.594813
4,1.341600,1.274155,15.388976,0.231839,3.575672


There were missing keys in the checkpoint model loaded: ['encoder.embed_tokens.weight', 'decoder.embed_tokens.weight', 'lm_head.weight'].


TrainOutput(global_step=1600, training_loss=1.5947604560852051, metrics={'train_runtime': 3643.6624, 'train_samples_per_second': 7.041, 'train_steps_per_second': 0.439, 'total_flos': 913436835840000.0, 'train_loss': 1.5947604560852051, 'epoch': 4.989280841941142})

In [14]:
# Save the model to Google Drive (specific to Google Colab)
from google.colab import drive
drive.mount('/content/drive')
model.save_pretrained("/content/drive/My Drive/linux_command_translator_base")
tokenizer.save_pretrained("/content/drive/My Drive/linux_command_translator_base")


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


('/content/drive/My Drive/linux_command_translator_base/tokenizer_config.json',
 '/content/drive/My Drive/linux_command_translator_base/special_tokens_map.json',
 '/content/drive/My Drive/linux_command_translator_base/spiece.model',
 '/content/drive/My Drive/linux_command_translator_base/added_tokens.json')

In [18]:
# Function to load model and tokenizer
def load_model_from_path(model_path):
    tokenizer = T5Tokenizer.from_pretrained(model_path)
    model = T5ForConditionalGeneration.from_pretrained(model_path)
    return tokenizer, model

In [19]:
# Function to translate natural language to command
def translate_to_command(input_text, tokenizer, model, device='cpu'):
    model.to(device)
    input_ids = tokenizer.encode("translate English to Bash: " + input_text, return_tensors="pt").to(device)
    outputs = model.generate(input_ids, max_length=50, num_beams=5, early_stopping=True)
    command = tokenizer.decode(outputs[0], skip_special_tokens=True)
    return command

# Assuming you have saved your model to the path below
model_path = '/content/drive/My Drive/linux_command_translator_base'

# Load model and tokenizer
tokenizer, model = load_model_from_path(model_path)

# Load dataset and select the 'train' split
dataset = load_dataset("neulab/tldr", split='train')

# Iterate over the first 10 entries of the train split
for i in range(100):
    input_text = dataset[i]['nl']  # Natural language instruction
    actual_command = dataset[i]['cmd']  # Corresponding command
    predicted_command = translate_to_command(input_text, tokenizer, model)

    print(f"Input Text: {input_text}")
    print(f"Actual Command: {actual_command}")
    print(f"Predicted Command: {predicted_command}")
    print("-" * 80)  # Separator for readability

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.
/usr/local/lib/python3.10/dist-packages/datasets/load.py:1486: FutureWarning: The repository for neulab/tldr contains custom code which must be executed to correctly load the dataset. You can inspect the repository content at https://hf.co/datasets/neulab/tldr
You can avoid this message in future by passing the argument `trust_remote_code=True`.
Passing `trust_remote_code=True` will be mandatory to load this dataset from the next major release of `datasets`.
  warnings.warn(


Input Text: get the label of a fat32 partition
Actual Command: fatlabel {{/dev/sda1}}
Predicted Command: fat32 -l path/to/partition
--------------------------------------------------------------------------------
Input Text: set the label of a fat32 partition
Actual Command: fatlabel {{/dev/sdc3}} "{{new_label}}"
Predicted Command: fat32 -l path/to/partition
--------------------------------------------------------------------------------
Input Text: search for a package in your current sources
Actual Command: apt-cache search {{query}}
Predicted Command: git search package_name
--------------------------------------------------------------------------------
Input Text: show information about a package
Actual Command: apt-cache show {{package}}
Predicted Command: git show package_name
--------------------------------------------------------------------------------
Input Text: show whether a package is installed and up to date
Actual Command: apt-cache policy {{package}}
Predicted Comman

In [20]:
# Load dataset and select the 'train' split
dataset = load_dataset("neulab/tldr", split='validation')

# Iterate over the first 10 entries of the train split
for i in range(100):
    input_text = dataset[i]['nl']  # Natural language instruction
    actual_command = dataset[i]['cmd']  # Corresponding command
    predicted_command = translate_to_command(input_text, tokenizer, model)

    print(f"Input Text: {input_text}")
    print(f"Actual Command: {actual_command}")
    print(f"Predicted Command: {predicted_command}")
    print("-" * 80)  # Separator for readability

Input Text: generate ascii art for a given text
Actual Command: toilet {{input_text}}
Predicted Command: ascii art text
--------------------------------------------------------------------------------
Input Text: generate ascii art using a custom font file
Actual Command: toilet {{input_text}} -f {{font_filename}}
Predicted Command: ascii art path/to/file.ascii
--------------------------------------------------------------------------------
Input Text: generate ascii art using a filter
Actual Command: toilet {{input_text}} --filter {{filter_name}}
Predicted Command: ascii -f path/to/file
--------------------------------------------------------------------------------
Input Text: show available toilet filters
Actual Command: toilet --filter list
Predicted Command: toilet filter
--------------------------------------------------------------------------------
Input Text: start an application with the procfile in the current directory
Actual Command: foreman start
Predicted Command: pcstar

In [21]:
# Load dataset and select the 'train' split
dataset = load_dataset("neulab/tldr", split='test')

# Iterate over the first 10 entries of the train split
for i in range(100):
    input_text = dataset[i]['nl']  # Natural language instruction
    actual_command = dataset[i]['cmd']  # Corresponding command
    predicted_command = translate_to_command(input_text, tokenizer, model)

    print(f"Input Text: {input_text}")
    print(f"Actual Command: {actual_command}")
    print(f"Predicted Command: {predicted_command}")
    print("-" * 80)  # Separator for readability

Input Text: delete a shared memory segment by id
Actual Command: ipcrm --shmem-id {{shmem_id}}
Predicted Command: id delete path/to/shared_memory
--------------------------------------------------------------------------------
Input Text: delete a shared memory segment by key
Actual Command: ipcrm --shmem-key {{shmem_key}}
Predicted Command: ssh delete path/to/key
--------------------------------------------------------------------------------
Input Text: delete an ipc queue by id
Actual Command: ipcrm --queue-id {{ipc_queue_id}}
Predicted Command: ipc queue delete id
--------------------------------------------------------------------------------
Input Text: delete an ipc queue by key
Actual Command: ipcrm --queue-key {{ipc_queue_key}}
Predicted Command: ipc-delete
--------------------------------------------------------------------------------
Input Text: delete a semaphore by id
Actual Command: ipcrm --semaphore-id {{semaphore_id}}
Predicted Command: semaphore delete semaphore_id
--